In [1]:
import polars as pl

In [77]:
import pytz
import datetime
import pandas as pd

test = pl.read_csv("./assets/bq-results-20230125-202210-1674678181880.csv")
# print(pytz.timezone(test['timezone_str'].iat[0]).localize(datetime.datetime.now()).strftime("%z"))
# pytz.timezone(test.loc[test['timezone_str']]).localize(datetime.datetime.now()).strftime("%z")
# temp_df = (
#     test.groupby("store_id")["timezone_str"]
#     .apply(lambda x: print(type(x)))
#     .reset_index()
# )
# f = test.groupby("store_id")['timezone_str'].apply(lambda x: pytz.timezone(x.iloc[0]).localize(datetime.datetime(2023, 1, 21)).strftime("%z"))
time_zone_df = test.select(
    pl.col('store_id'),
    pl.col("timezone_str").apply(
        lambda x: pytz.timezone(x).localize(datetime.datetime(2023, 1, 21)).utcoffset()
    ),
)
time_zone_df.filter(
  pl.col('timezone_str') == ""
)

store_id,timezone_str
i64,duration[μs]


In [42]:
store_status_lazy = pl.scan_csv("./assets/store status.csv")

open_hours_df = pl.scan_csv("./assets/Menu hours.csv")
# time_zone_df = pl.scan_csv("./assets/bq-results-20230125-202210-1674678181880.csv")
final = (
    store_status_lazy.lazy()
    .join(open_hours_df.lazy(), how="left", left_on="store_id", right_on="store_id")
    .join(time_zone_df.lazy(), how="left", left_on="store_id", right_on="store_id")
)
# final.collect(streaming=True)

In [34]:
pytz.timezone("America/Chicago").localize(datetime.datetime(2023, 1, 21)).utcoffset()

datetime.timedelta(days=-1, seconds=64800)

In [79]:

no_null_final = (
    final.lazy()
    .with_columns(
        [
            pl.col("day").fill_null(6),
            pl.col("start_time_local")
            .fill_null("00:00:00")
            .str.strptime(pl.Datetime, format="%H:%M:%S"),
            pl.col("end_time_local")
            .fill_null("23:59:59")
            .str.strptime(pl.Datetime, format="%H:%M:%S"),
            pl.col("timezone_str").fill_null(datetime.timedelta(hours=-6)),
            pl.col("timestamp_utc")
            .str.replace(" UTC", "")
            .str.strptime(pl.Datetime, format="%F %H:%M:%S%.6f")
            .dt.replace_time_zone("UTC"),
        ]
    )
    .lazy()
    .filter((pl.col("status").is_not_null()) & (pl.col("timestamp_utc").is_not_null()))
    .lazy()
    .select(
        [
            pl.col("*"),
            (
                pl.col("start_time_local").dt.year()
                + pl.col("timestamp_utc").dt.year()
                - 1
            ).alias("year_start"),
            (
                pl.col("start_time_local").dt.month()
                + pl.col("timestamp_utc").dt.month()
                - 1
            ).alias("month_start"),
            (
                pl.col("start_time_local").dt.day()
                + pl.col("timestamp_utc").dt.day()
                - 1
            ).alias("day_start"),
            (
                pl.col("end_time_local").dt.year()
                + pl.col("timestamp_utc").dt.year()
                - 1
            ).alias("year_end"),
            (
                pl.col("end_time_local").dt.month()
                + pl.col("timestamp_utc").dt.month()
                - 1
            ).alias("month_end"),
            (
                pl.col("end_time_local").dt.day() + pl.col("timestamp_utc").dt.day() - 1
            ).alias("day_end"),
        ]
    )
    .lazy()
    .select(
        [
            pl.col(["store_id", "status", "timestamp_utc", "day"]),
            pl.datetime(
                "year_start",
                "month_start",
                "day_start",
                pl.col("start_time_local").dt.strftime(format="%H"),
                pl.col("start_time_local").dt.strftime(format="%M"),
                pl.col("start_time_local").dt.strftime(format="%S"),
            )
            .dt.strftime("%Y-%m-%d %H:%M:%S")
            .str.strptime(pl.Datetime, format="%F %H:%M:%S")
            .alias("start_time_local"),
            pl.datetime(
                "year_end",
                "month_end",
                "day_end",
                pl.col("end_time_local").dt.strftime(format="%H"),
                pl.col("end_time_local").dt.strftime(format="%M"),
                pl.col("end_time_local").dt.strftime(format="%S"),
            )
            .dt.strftime("%Y-%m-%d %H:%M:%S")
            .str.strptime(pl.Datetime, format="%F %H:%M:%S")
            .alias("end_time_local"),
            pl.col("timezone_str"),
        ]
    )
)
no_null_final.fetch(n_rows=10)
# print(another_test.select(pl.col('start_time_local')).collect(streaming=True))

store_id,status,timestamp_utc,day,start_time_local,end_time_local,timezone_str
i64,str,"datetime[μs, UTC]",i64,datetime[μs],datetime[μs],duration[μs]
8419537941919820732,"""active""",2023-01-22 12:09:39.388884 UTC,6,2023-01-22 00:00:00,2023-01-22 23:59:59,-6h
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,6,2023-01-24 00:00:00,2023-01-24 23:59:59,-6h
8377465688456570187,"""active""",2023-01-24 09:07:26.441407 UTC,6,2023-01-24 00:00:00,2023-01-24 23:59:59,-6h
5955337179846162144,"""active""",2023-01-24 09:08:07.634899 UTC,6,2023-01-24 00:00:00,2023-01-24 23:59:59,-6h
1169347689335808384,"""active""",2023-01-24 09:08:18.436854 UTC,6,2023-01-24 00:00:00,2023-01-24 23:59:59,-6h
3739729523266121270,"""active""",2023-01-24 09:08:23.138922 UTC,6,2023-01-24 00:00:00,2023-01-24 23:59:59,-6h
4430370444829587327,"""active""",2023-01-24 09:09:37.456563 UTC,6,2023-01-24 00:00:00,2023-01-24 23:59:59,-6h
1912555533282230523,"""active""",2023-01-24 09:09:58.684489 UTC,6,2023-01-24 00:00:00,2023-01-24 23:59:59,-6h
5489547402882838709,"""active""",2023-01-24 09:10:28.873707 UTC,6,2023-01-24 00:00:00,2023-01-24 23:59:59,-6h


In [80]:
# time_zone_df.collect(streaming=True)
# pandas_df = no_null_final.collect().to_pandas()
another_test = no_null_final.select(
    pl.col(['store_id', 'status', 'timestamp_utc', 'day']),
    (pl.col("start_time_local") - pl.col('timezone_str')).dt.replace_time_zone("UTC"),
    (pl.col("end_time_local") - pl.col('timezone_str')).dt.replace_time_zone("UTC"),
    pl.col('timezone_str')
).drop('timezone_str')

another_test.collect()
# print(another_test.collect(streaming=True))
# print(another_test.fetch(n_rows=19))
# converted = another_test.with_columns(
#     pl.col('start_time_local').dt.replace_time_zone(time_zone='UTC').alias('start_time_converted')
# )
# converted.fetch(n_rows=10)

store_id,status,timestamp_utc,day,start_time_local,end_time_local
i64,str,"datetime[μs, UTC]",i64,"datetime[μs, UTC]","datetime[μs, UTC]"
8419537941919820732,"""active""",2023-01-22 12:09:39.388884 UTC,6,2023-01-22 06:00:00 UTC,2023-01-23 05:59:59 UTC
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,0,2023-01-24 13:00:00 UTC,2023-01-24 23:30:00 UTC
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,6,2023-01-24 13:00:00 UTC,2023-01-24 23:30:00 UTC
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,5,2023-01-24 13:00:00 UTC,2023-01-24 23:30:00 UTC
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,2,2023-01-24 13:00:00 UTC,2023-01-24 23:30:00 UTC
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,3,2023-01-24 13:00:00 UTC,2023-01-24 23:30:00 UTC
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,4,2023-01-24 13:00:00 UTC,2023-01-24 23:30:00 UTC
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,1,2023-01-24 13:00:00 UTC,2023-01-24 23:30:00 UTC
8377465688456570187,"""active""",2023-01-24 09:07:26.441407 UTC,2,2023-01-24 17:00:00 UTC,2023-01-25 05:30:00 UTC


In [31]:
f = pandas_df['timezone_str'].apply(lambda x: pytz.timezone(x).localize(datetime.datetime(2023, 1, 21)).strftime("%z"))
# f.reset_index()
p = pl.from_pandas(f.reset_index().sort_values('store_id', ascending=True))

KeyboardInterrupt: 

In [30]:

timezone_str = p.select(pl.col('timezone_str')).to_series()
print(type(timezone_str))
no_null_final.sort('store_id', descending=False).collect().replace('timezone_str', timezone_str)

<class 'polars.series.series.Series'>


ShapeError: resulting series has length 14092 while the dataframe has height 11823128

In [3]:
no_null_final = (
    final.lazy()
    .with_columns(
        [
            pl.col("day").fill_null(6),
            pl.col("start_time_local")
            .fill_null("00:00:00")
            .str.strptime(pl.Datetime, format="%H:%M:%S"),
            pl.col("end_time_local")
            .fill_null("23:59:59")
            .str.strptime(pl.Datetime, format="%H:%M:%S"),
            pl.col("timezone_str").fill_null("America/Chicago"),
            pl.col("timestamp_utc")
            .str.replace(" UTC", "")
            .str.strptime(pl.Datetime, format="%F %H:%M:%S%.6f")
            .dt.replace_time_zone("UTC"),
        ]
    )
    .lazy()
    .filter((pl.col("status").is_not_null()) & (pl.col("timestamp_utc").is_not_null()))
    .lazy()
    .select(
        [
            pl.col("*"),
            (
                pl.col("start_time_local").dt.year()
                + pl.col("timestamp_utc").dt.year()
                - 1
            ).alias("year_start"),
            (
                pl.col("start_time_local").dt.month()
                + pl.col("timestamp_utc").dt.month()
                - 1
            ).alias("month_start"),
            (
                pl.col("start_time_local").dt.day()
                + pl.col("timestamp_utc").dt.day()
                - 1
            ).alias("day_start"),
            (
                pl.col("end_time_local").dt.year()
                + pl.col("timestamp_utc").dt.year()
                - 1
            ).alias("year_end"),
            (
                pl.col("end_time_local").dt.month()
                + pl.col("timestamp_utc").dt.month()
                - 1
            ).alias("month_end"),
            (
                pl.col("end_time_local").dt.day() + pl.col("timestamp_utc").dt.day() - 1
            ).alias("day_end"),
        ]
    )
    .lazy()
    .select(
        [
            pl.col(["store_id", "status", "timestamp_utc", "day"]),
            pl.datetime(
                "year_start",
                "month_start",
                "day_start",
                pl.col("start_time_local").dt.strftime(format="%H"),
                pl.col("start_time_local").dt.strftime(format="%M"),
                pl.col("start_time_local").dt.strftime(format="%S"),
            )
            .dt.strftime("%Y-%m-%d %H:%M:%S")
            .str.strptime(pl.Datetime, format="%F %H:%M:%S")
            .alias("start_time_local"),
            pl.datetime(
                "year_end",
                "month_end",
                "day_end",
                pl.col("end_time_local").dt.strftime(format="%H"),
                pl.col("end_time_local").dt.strftime(format="%M"),
                pl.col("end_time_local").dt.strftime(format="%S"),
            )
            .dt.strftime("%Y-%m-%d %H:%M:%S")
            .str.strptime(pl.Datetime, format="%F %H:%M:%S")
            .alias("end_time_local"),
            pl.col("timezone_str"),
        ]
    )
)
# no_null_final.fetch(n_rows=10)

In [25]:
hello = no_null_final.collect().to_pandas()

In [4]:
from io import StringIO
from csv import writer
import pandas as pd
import datetime
# (8419537941919820732, 'active', datetime.datetime(2023, 1, 22, 12, 9, 39, 388884, tzinfo=zoneinfo.ZoneInfo(key='UTC')), 6, datetime.datetime(2023, 1, 22, 0, 0), datetime.datetime(2023, 1, 22, 23, 59, 59), 'America/Chicago')
output = StringIO()
collected = no_null_final.collect()
csv_writer = writer(output)
for value in collected.iter_rows():
    # print(value)
    csv_writer.writerow(value)

In [4]:
no_null_final = no_null_final.filter(
  (pl.col('status').is_not_null()) & (pl.col('timestamp_utc').is_not_null())
)

In [5]:
temp = no_null_final.lazy().select(
    [
        pl.col("*"),
        (
            pl.col("start_time_local").dt.year() + pl.col("timestamp_utc").dt.year() - 1
        ).alias("year_start"),
        (
            pl.col("start_time_local").dt.month()
            + pl.col("timestamp_utc").dt.month()
            - 1
        ).alias("month_start"),
        (
            pl.col("start_time_local").dt.day() + pl.col("timestamp_utc").dt.day() - 1
        ).alias("day_start"),
        (
            pl.col("end_time_local").dt.year() + pl.col("timestamp_utc").dt.year() - 1
        ).alias("year_end"),
        (
            pl.col("end_time_local").dt.month() + pl.col("timestamp_utc").dt.month() - 1
        ).alias("month_end"),
        (
            pl.col("end_time_local").dt.day() + pl.col("timestamp_utc").dt.day() - 1
        ).alias("day_end")
    ]
)

h = temp.lazy().select(
    [
        pl.col(['store_id', 'status', 'timestamp_utc', 'day']),
        pl.datetime(
            "year_start",
            "month_start",
            "day_start",
            pl.col("start_time_local").dt.strftime(format="%H"),
            pl.col("start_time_local").dt.strftime(format="%M"),
            pl.col("start_time_local").dt.strftime(format="%S"),
        )
        .dt.strftime("%Y-%m-%d %H:%M:%S")
        .str.strptime(pl.Datetime, format="%F %H:%M:%S")
        .alias("start_time_local"),
        pl.datetime(
            "year_end",
            "month_end",
            "day_end",
            pl.col("end_time_local").dt.strftime(format="%H"),
            pl.col("end_time_local").dt.strftime(format="%M"),
            pl.col("end_time_local").dt.strftime(format="%S"),
        )
        .dt.strftime("%Y-%m-%d %H:%M:%S")
        .str.strptime(pl.Datetime, format="%F %H:%M:%S")
        .alias("end_time_local"),
        pl.col("timezone_str")
    ]
)
# h.fetch(n_rows=10)

# k.drop(["year_start", "month_start", "day_start", "year_end", "month_end", "day_end"])

In [13]:
pl.SQLContext(frame=no_null_final).execute(
  '''SELECT timezone_str, ,   
    OrderDate AT TIME ZONE 'Eastern Standard Time' AS OrderDate_TimeZoneEST,  
    OrderDate AT TIME ZONE 'Eastern Standard Time'   
    AT TIME ZONE 'UTC' AS OrderDate_TimeZoneUTC  
FROM Sales.SalesOrderHeader'''
).collect(streaming=True)

ComputeError: sql parser error: Expected end of statement, found: (

In [66]:
import pandas as pd

pandas_df = (
    h.lazy()
    .select(pl.col("*"))
    .collect()
    .to_pandas()
)
timezones = pandas_df["timezone_str"].unique()
pandas_df["start_time_local"] = pd.to_datetime(pandas_df["start_time_local"])
pandas_df["end_time_local"] = pd.to_datetime(pandas_df['end_time_local'])
# for timezone in timezones:
#     pandas_df.loc[pandas_df["timezone_str"] == timezone, "start_time_local"] = (
#         pandas_df.loc[pandas_df["timezone_str"] == timezone, "start_time_local"]
#         .dt.tz_localize("UTC")
#         .dt.tz_convert(timezone)
#     )

In [ ]:
pandas_df.head()

In [67]:
temp_df = (
    pandas_df.groupby("timezone_str")["start_time_local"]
    .apply(lambda x: x.dt.tz_localize(x.name).dt.tz_convert("UTC"))
    .reset_index()
)
temp_df['end_time_local'] = (
    pandas_df.groupby("timezone_str")["end_time_local"]
    .apply(lambda x: x.dt.tz_localize(x.name).dt.tz_convert("UTC"))
    .reset_index()['end_time_local']
)
# temp_df.sort_values(by='level_1', ascending=True, inplace=True)
new_df = pandas_df.merge(temp_df, how='left', left_index=True, right_on='level_1')
# # temp_df.sort_values(by='level_1', ascending=True, inplace=True)
# # pandas_df["converted"] = (temp_df['start_time_local'])
# # pandas_df['index_original'] = temp_df['level_1']
new_polar = pl.from_pandas(new_df)
# pd.merge(
#     pandas_df, new_df["start_time_local"], left_index=True, right_on=new_df["level_1"], how="left"
# )

In [68]:
# new_polar = new_polar.lazy().select(
#   pl.col(['start_time_local', 'end_time_local']),
#   pl.col('level_1').cast(pl.UInt32)
# )
# h.lazy().join(new_polar.lazy(), left_on='row_nr', right_on='level_1', how='outer').fetch(n_rows=10)
new_polar = new_polar.lazy().drop(
    ["start_time_local_x", "end_time_local_x", "level_1", "timezone_str_x"]
).lazy().rename(
    {
        "start_time_local_y": "start_time_local",
        "end_time_local_y": "end_time_local",
        "timezone_str_y": "timezone_str",
    }
)


In [19]:
new_polar.fetch(n_rows=10)

store_id,status,timestamp_utc,day,timezone_str,start_time_local,end_time_local
i64,str,"datetime[ns, UTC]",i64,str,"datetime[ns, UTC]","datetime[ns, UTC]"
8419537941919820732,"""active""",2023-01-22 12:09:39.388884 UTC,6,"""America/Chicag…",2023-01-22 06:00:00 UTC,2023-01-23 05:59:59 UTC
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,0,"""America/New_Yo…",2023-01-24 13:00:00 UTC,2023-01-24 23:30:00 UTC
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,6,"""America/New_Yo…",2023-01-24 13:00:00 UTC,2023-01-24 23:30:00 UTC
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,5,"""America/New_Yo…",2023-01-24 13:00:00 UTC,2023-01-24 23:30:00 UTC
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,2,"""America/New_Yo…",2023-01-24 13:00:00 UTC,2023-01-24 23:30:00 UTC
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,3,"""America/New_Yo…",2023-01-24 13:00:00 UTC,2023-01-24 23:30:00 UTC
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,4,"""America/New_Yo…",2023-01-24 13:00:00 UTC,2023-01-24 23:30:00 UTC
54515546588432327,"""active""",2023-01-24 09:06:42.605777 UTC,1,"""America/New_Yo…",2023-01-24 13:00:00 UTC,2023-01-24 23:30:00 UTC
8377465688456570187,"""active""",2023-01-24 09:07:26.441407 UTC,2,"""America/Los_An…",2023-01-24 17:00:00 UTC,2023-01-25 05:30:00 UTC


In [ ]:
# h.lazy().join(new_polar.lazy(), on=['store_id', 'start_time_local'], how='left').fetch(n_rows=10)
# h.fetch(n_rows=10)
# lazy_new_polar = new_polar.lazy().with_columns(
#     [
#         pl.col(["store_id", "timezone_str"]),
#         (pl.col("start_time_local").cast(int) // 1000)
#         .cast(pl.Datetime)
#         .dt.cast_time_unit("us"),
#         (pl.col("converted").cast(int) // 1000)
#         .cast(pl.Datetime)
#         .dt.cast_time_unit("us"),
#     ]
# )
# h.lazy().join(lazy_new_polar.lazy(), on=['start_time_local', 'timezone_str'], how='left').fetch(n_rows=10)
new_polar.sort(by="level_1", descending=False)
converted_series = new_polar.lazy().select(
  pl.col('start_time_local')
)
h.lazy().with_columns(
  converted_series.collect()
).collect()

In [ ]:
no_null_final_collected = no_null_final.collect(streaming=True)


In [ ]:
seperated_df = h.collect().partition_by(by='store_id', as_dict=True)

In [ ]:
print(seperated_df)

In [ ]:
import datetime
import pytz

def convert_to_utc(
    timezone_str: str, date: datetime.datetime, sample_date: datetime.datetime
) -> datetime.datetime:
    # sample_date is important as the result varies as the month changes
    date = datetime.datetime(date).replace(
        year=sample_date.year, day=sample_date.day, month=sample_date.month
    )
    tz = pytz.timezone(timezone_str)
    local_aware_time = tz.normalize(tz.localize(date, is_dst=False))
    as_utc = local_aware_time.astimezone(pytz.UTC)
    return as_utc


In [ ]:
list_keys = list(seperated_df.keys())
new_dict = {}
for i in list_keys:
    sample_timezone = seperated_df[i].select("timezone_str").head(1)["timezone_str"][0]
    # sample_timezone = seperated_df['timezone_str'].iat[index]
    # tz = pytz.timezone(sample_timezone)
    # local_aware_time = tz.normalize(tz.localize(seperated_df['start_time_local'].iat[index]))
    # as_utc = local_aware_time.astimezone(pytz.UTC)
    # seperated_df.loc[index, 'start_time_local'] = as_utc
    new_dict[i] = (
        seperated_df[i]
        .lazy()
        .select(
            pl.col(["store_id", "status", "timestamp_utc", "day"]),
            pl.col("start_time_local")
            .dt.replace_time_zone(sample_timezone)
            .dt.convert_time_zone("UTC"),
            pl.col("end_time_local")
            .dt.replace_time_zone(sample_timezone)
            .dt.convert_time_zone("UTC"),
            pl.col(["timezone_str"]),
        )
    )
    # ).collect()
    # temp.filter(
    #     pl.col("start_time_local")
    # )

In [ ]:
new_dict[4845598456063235048].collect(streaming=True)

In [ ]:
# handles 24 hours only
delta = new_dict[4845598456063235048].filter(
    pl.col("end_time_local") - pl.col("start_time_local")
    == pl.duration(hours=23, minutes=59, seconds=59)
)
next_step = delta.filter(pl.col("end_time_local") >= pl.col("timestamp_utc"))
next_step = next_step.filter(pl.col("start_time_local") <= pl.col("timestamp_utc"))
next_step = next_step.filter(pl.col("day") == pl.col("timestamp_utc").dt.weekday() - 1)
active = next_step.filter(pl.col('status') == 'active')
active.select(pl.count()).collect(streaming=True)

In [84]:
filter_247 = another_test.lazy().filter(
    (
        pl.col("end_time_local") - pl.col("start_time_local")
        == pl.duration(hours=23, minutes=59, seconds=59)
    )
    & (
        (
            (pl.col("end_time_local") >= pl.col("timestamp_utc"))
            & (pl.col("start_time_local") <= pl.col("timestamp_utc"))
            | (
                (pl.col("start_time_local") >= pl.col("timestamp_utc"))
                & (pl.col("end_time_local") <= pl.col("timestamp_utc"))
            )
        )
    )
    & (pl.col("day") == (pl.col("timestamp_utc").dt.weekday() - 1))
)

filter_non_247 = another_test.lazy().filter(
    (
        pl.col("end_time_local") - pl.col("start_time_local")
        != pl.duration(hours=23, minutes=59, seconds=59)
    )
    & (
        (
            (pl.col("end_time_local") >= pl.col("timestamp_utc"))
            & (pl.col("start_time_local") <= pl.col("timestamp_utc"))
        )
    )
    & (pl.col("day") == (pl.col("timestamp_utc").dt.weekday() - 1))
)
# filtered_frame = filter_non_247.collect().extend(filter_247.collect())
# filtered_frame = filter_non_247.collect().vstack(filter_247.collect())
filtered_frame = pl.concat([filter_247, filter_non_247], parallel=True)
# filter_non_247.fetch(n_rows=10000)
# )
#     (pl.col("end_time_local") <= pl.col("start_time_local"))
# )
# # filter_non_247.select(
# #     pl.count()
# # ).collect()
# some_filter.select(
#     pl.count()
# ).collect()
# test_filter = new_dict[4845598456063235048].filter(
#     pl.col('end_time_local') > pl.col('start_time_local')
# )
# new_dict[4845598456063235048].select(
#     pl.count()
# ).collect()
# filtered_frame
# 610_616
# filtered_frame.collect()

In [11]:
filtered_frame.collect(streaming=True)

PanicException: called `Option::unwrap()` on a `None` value

In [10]:

filter_non_247 = filter_non_247.sort(by='timestamp_utc', descending=False)
# test = filter_non_247.collect().partition_by(by='store_id', as_dict=True)
# test.keys()

In [11]:
import pandas as pd
from io import StringIO
from csv import writer
import pandas as pd
import numpy as np
import datetime

for value in list(test.keys()):
    pandas_df = test[value].to_pandas()
    pandas_df.sort_values("timestamp_utc", ascending=True, inplace=True)
    ts = pd.Series(index=pandas_df["timestamp_utc"])
    ts = ts.resample("T").mean()
    ts.interpolate(method="spline", order=3)
    ts = ts.reset_index()
    output = StringIO()
    csv_writer = writer(output)
    header = False
    counter = 0

    # for index, row in temp.iterrows():
    #     if(not header):
    #       csv_writer.writerow(temp)
    #       header = True

    #     csv_writer.writerow(row)
    #     counter += 1
    current_store_id = np.nan
    current_status = np.nan
    current_day = np.nan
    current_start_time = np.nan
    current_end_time = np.nan
    current_timezone = np.nan

    for value in ts["timestamp_utc"]:
        if not header:
            csv_writer.writerow(pandas_df)
            header = True
        next_value = pandas_df["timestamp_utc"].iat[counter]
        if value == next_value.replace(second=0, microsecond=0, nanosecond=0):
            current_store_id = pandas_df["store_id"].iat[counter]
            current_status = pandas_df["status"].iat[counter]
            current_day = pandas_df["day"].iat[counter]
            current_start_time = pandas_df["start_time_local"].iat[counter]
            current_end_time = pandas_df["end_time_local"].iat[counter]
            current_timezone = pandas_df["timezone_str"].iat[counter]
            # temp_dict = {
            #     "store_id": current_store_id,
            #     "status": current_status,
            #     "timestamp_utc": value,
            #     "day": current_day,
            #     "start_time_local": current_start_time,
            #     "end_time_local": current_end_time,
            #     "timezone_str": current_timezone,
            # }
            temp_list = [
                current_store_id,
                current_status,
                value,
                current_day,
                current_timezone,
                current_start_time,
                current_end_time,
            ]
            csv_writer.writerow(temp_list)
            counter += 1
        else:
            # temp_dict = {
            #     "store_id": current_store_id,
            #     "status": current_status,
            #     "timestamp_utc": value,
            #     "day": current_day,
            #     "start_time_local": current_start_time,
            #     "end_time_local": current_end_time,
            #     "timezone_str": current_timezone,
            # }
            temp_list = [
                current_store_id,
                current_status,
                value,
                current_day,
                current_timezone,
                current_start_time,
                current_end_time,
            ]
            csv_writer.writerow(temp_list)

    output.seek(0)  # we need to get back to the start of the BytesIO
    df = pd.read_csv(output)
    df.sort_values("timestamp_utc", ascending=False, inplace=True)
    current_date = df["timestamp_utc"].iat[0]
    polars_df = pl.from_pandas(df)
    polars_df = polars_df.lazy().with_columns(
        [
            pl.col(["store_id", "status"]),
            pl.col("timestamp_utc").str.strptime(
                pl.Datetime, format="%F %H:%M:%S%.f%z"
            ),
            pl.col("day"),
            pl.col("start_time_local").str.strptime(
                pl.Datetime, format="%F %H:%M:%S%z"
            ),
            pl.col("end_time_local").str.strptime(pl.Datetime, format="%F %H:%M:%S%z"),
        ]
    )
    polars_df = polars_df.lazy().with_columns(
        [
            pl.col(["store_id", "status"]),
            # (pl.col("start_time_local").cast(int) // 1000)
            #         .cast(pl.Datetime)
            #         .dt.cast_time_unit("us"),
            (pl.col("timestamp_utc").cast(int) // 1000)
            .cast(pl.Datetime)
            .dt.cast_time_unit("us")
            .dt.replace_time_zone("UTC"),
            pl.col(["day", "timezone_str", "start_time_local", "end_time_local"]),
        ]
    )

NameError: name 'test' is not defined

In [47]:
filter_non_247 = filter_non_247.sort(by='timestamp_utc', descending=True)
test = filter_non_247.collect().partition_by(by='store_id', as_dict=True, maintain_order=True)

In [85]:
wow = filtered_frame.groupby("store_id").agg(pl.col(['timestamp_utc', 'status', 'day']))
wow

<LazyFrame [4 cols, {"store_id": Int64 … "day": List(Int64)}] at 0x1F3B6F1B490>

In [89]:
middle = filtered_frame.groupby("store_id").agg(pl.col(['timestamp_utc', 'status', 'day'])).select(
    pl.all(),
    pl.col("timestamp_utc").list.max().alias("max date"),
).select(
    pl.all(),
    (
        ((pl.col("max date") - datetime.timedelta(days=7)).cast(int))
        .cast(pl.Datetime)
        .dt.cast_time_unit("ns")
        .dt.replace_time_zone("UTC")
    ).alias("till date"),
).explode(pl.col(["timestamp_utc", "status", "day"])).select(
    pl.all(), (pl.col("timestamp_utc").dt.weekday() - 1).cast(pl.Int64).alias("weekday")
).collect()
unique_store_ids = middle.select(pl.col("store_id").unique())

In [62]:
import datetime
amazing = wow.select(
    pl.all(),
    pl.col("timestamp_utc").list.max().alias("max date"),
)
really_amazing = amazing.select(
    pl.all(),
    (
        ((pl.col("max date") - datetime.timedelta(days=7)).cast(int))
        .cast(pl.Datetime)
        .dt.cast_time_unit("ns")
        .dt.replace_time_zone("UTC")
    ).alias("till date"),
)
f = really_amazing.explode(pl.col(["timestamp_utc", "status", "day"]))
# temp = f.select(pl.col('timestamp_utc').dt.weekday()-1)
middle = f.select(
    pl.all(), (pl.col("timestamp_utc").dt.weekday() - 1).cast(pl.Int64).alias("weekday")
)
unique_store_ids = middle.select(pl.col("store_id").unique())
unique_store_ids.collect()
# z = middle.filter(
#     (
#         (pl.col("timestamp_utc") > pl.col("till date"))
#         & (pl.col("timestamp_utc") <= pl.col("max date"))
#     )
#     & (pl.col("weekday") == pl.col("day"))
# )
# whyareyoulikethis = z.groupby("store_id").agg(
#     ["timestamp_utc", "status", "day", "max date", "till date", "weekday"]
# )
# # z.groupby('store_id').agg(['timestamp_utc', 'status', 'day'])
# whyareyoulikethis.select(
#     pl.col("status").list.count_match("active").alias("active_count"),
#     pl.col("status").list.count_match("inactive").alias("inactive_count"),
# )

ComputeError: strict conversion from `str` to `i16` failed for value(s) ["-28800000000", "7200000000", … "-36000000000"]; if you were trying to cast Utf8 to temporal dtypes, consider using `strptime`

In [90]:
# finale = pl.DataFrame(
#     {
#         "store_id": [],
#         "timestamp_utc": [],
#         "status": [],
#         "day": [],
#         "max date": [],
#         "till date": [],
#         "weekday": [],
#     },
#     schema={
#         "store_id": pl.Int64,
#         "timestamp_utc": pl.Datetime(time_unit='ns', time_zone='UTC'),
#         "status": str,
#         "day": pl.Int64,
#         "max date": pl.Datetime(time_unit='ns', time_zone='UTC'),
#         "till date": pl.Datetime(time_unit='us', time_zone='UTC'),
#         "weekday": pl.Int64,
#     },
# )
finale = {}
active_minutes = 0
inactive_minutes = 0
for value in unique_store_ids.iter_rows():
    s = middle.filter(pl.col("store_id") == value[0])
    z = s.sort(by="timestamp_utc", descending=False)
    k = z.upsample(time_column="timestamp_utc", every="1m").fill_null(
        strategy="forward"
    )
    # pl.concat([finale, k], how="align")
    n = k.lazy().filter(
        (
            (pl.col("timestamp_utc") > pl.col("till date"))
            & (pl.col("timestamp_utc") <= pl.col("max date"))
        )
        & (pl.col("weekday") == pl.col("day"))
    )
    m = (
        n.lazy()
        .groupby("store_id")
        .agg(pl.col(["timestamp_utc", "status", "max date", "till date"]))
    )
    l = m.lazy().select(
        pl.all(),
        pl.col("status").list.count_match("active").alias("active_count"),
        pl.col("status").list.count_match("inactive").alias("inactive_count"),
    )
    collected_l = l.collect()
    # print(collected_l)
    
    try:
        active_minutes = pl.first(collected_l["active_count"])
    except IndexError:
        active_minutes = 0
    try:
        pl.first(collected_l["inactive_count"])
    except IndexError:
        inactive_minutes = 0
    finale[value[0]] = [
        active_minutes, inactive_minutes
    ]
    # print(active_minutes/(60*24))

In [91]:
for i in list(finale.keys()):
  if finale[i][0] + finale[i][1] < 1000:
    print(finale[i])

[0, 0]
[0, 0]
[426, 0]
[0, 0]
[0, 0]
[0, 0]
[427, 0]
[178, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[716, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[482, 0]
[0, 0]
[0, 0]
[0, 0]
[174, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[420, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[479, 0]
[0, 0]
[488, 0]
[482, 0]
[0, 0]
[0, 0]
[422, 0]
[430, 0]
[0, 0]
[419, 0]
[176, 0]
[0, 0]
[0, 0]
[479, 0]
[0, 0]
[0, 0]
[476, 0]
[0, 0]
[434, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[478, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[481, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[180, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[479, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[486, 0]
[420, 0]
[0, 0]
[423, 0]
[486, 0]
[174, 0]
[0, 0]
[0, 0]
[424, 0]
[478, 0]
[0, 0]
[0, 0]
[422, 0]
[480, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[0, 0]
[482, 0]
[0, 0]
[0, 0]
[0, 0]


In [39]:
h = z.groupby("store_id").agg(pl.col(['timestamp_utc', 'status', 'max date', 'till date']))
h = h.select(
  pl.col(['store_id', 'status', 'max date', 'till date']),
  pl.col('timestamp_utc').list.sort(descending=False),
)


InvalidOperationError: argument in operation 'upsample' is not explicitly sorted

- If your data is ALREADY sorted, set the sorted flag with: '.set_sorted()'.
- If your data is NOT sorted, sort the 'expr/series/column' first.
    

In [30]:
for value in list(test.keys()):
    test[value] = (
        test[value].sort(by='timestamp_utc', descending=False)
        .upsample(time_column="timestamp_utc", every="1m")
        .fill_null(strategy="forward")
    )

In [ ]:
from io import StringIO
from csv import writer
import pandas as pd
import numpy as np
import datetime

output = StringIO()
csv_writer = writer(output)
header = False
counter = 0

# for index, row in temp.iterrows():
#     if(not header):
#       csv_writer.writerow(temp)
#       header = True

#     csv_writer.writerow(row)
#     counter += 1
current_store_id = np.nan
current_status = np.nan
current_day = np.nan
current_start_time = np.nan
current_end_time = np.nan
current_timezone = np.nan

for value in ts["timestamp_utc"]:
    if not header:
        csv_writer.writerow(pandas_df)
        header = True
    next_value = pandas_df["timestamp_utc"].iat[counter]
    if value == next_value.replace(second=0, microsecond=0, nanosecond=0):
        current_store_id = pandas_df["store_id"].iat[counter]
        current_status = pandas_df["status"].iat[counter]
        current_day = pandas_df["day"].iat[counter]
        current_start_time = pandas_df["start_time_local"].iat[counter]
        current_end_time = pandas_df["end_time_local"].iat[counter]
        current_timezone = pandas_df["timezone_str"].iat[counter]
        # temp_dict = {
        #     "store_id": current_store_id,
        #     "status": current_status,
        #     "timestamp_utc": value,
        #     "day": current_day,
        #     "start_time_local": current_start_time,
        #     "end_time_local": current_end_time,
        #     "timezone_str": current_timezone,
        # }
        temp_list = [
            current_store_id,
            current_status,
            value,
            current_day,
            current_timezone,
            current_start_time,
            current_end_time,
        ]
        csv_writer.writerow(temp_list)
        counter += 1
    else:
        # temp_dict = {
        #     "store_id": current_store_id,
        #     "status": current_status,
        #     "timestamp_utc": value,
        #     "day": current_day,
        #     "start_time_local": current_start_time,
        #     "end_time_local": current_end_time,
        #     "timezone_str": current_timezone,
        # }
        temp_list = [
            current_store_id,
            current_status,
            value,
            current_day,
            current_timezone,
            current_start_time,
            current_end_time,
        ]
        csv_writer.writerow(temp_list)

output.seek(0)  # we need to get back to the start of the BytesIO
df = pd.read_csv(output)
df.sort_values("timestamp_utc", ascending=False, inplace=True)
current_date = df["timestamp_utc"].iat[0]
polars_df = pl.from_pandas(df)
polars_df = polars_df.lazy().with_columns(
    [
        pl.col(["store_id", "status"]),
        pl.col("timestamp_utc").str.strptime(pl.Datetime, format="%F %H:%M:%S%.f%z"),
        pl.col("day"),
        pl.col("start_time_local").str.strptime(pl.Datetime, format="%F %H:%M:%S%z"),
        pl.col("end_time_local").str.strptime(pl.Datetime, format="%F %H:%M:%S%z"),
    ]
)
polars_df = polars_df.lazy().with_columns(
    [
        pl.col(["store_id", "status"]),
        # (pl.col("start_time_local").cast(int) // 1000)
        #         .cast(pl.Datetime)
        #         .dt.cast_time_unit("us"),
        (pl.col("timestamp_utc").cast(int) // 1000)
        .cast(pl.Datetime)
        .dt.cast_time_unit("us").dt.replace_time_zone("UTC"),
        pl.col(['day', 'timezone_str', 'start_time_local', 'end_time_local'])
    ]
)
polars_df

In [32]:
test[5652948553137053553]

timestamp_utc,store_id,status,day,timezone_str,start_time_local,end_time_local
"datetime[ns, UTC]",i64,str,i64,str,"datetime[ns, UTC]","datetime[ns, UTC]"
2023-01-18 05:00:03.523237 UTC,5652948553137053553,"""active""",2,"""America/New_Yo…",2023-01-18 05:00:00 UTC,2023-01-19 04:59:00 UTC
2023-01-18 05:01:03.523237 UTC,5652948553137053553,"""active""",2,"""America/New_Yo…",2023-01-18 05:00:00 UTC,2023-01-19 04:59:00 UTC
2023-01-18 05:02:03.523237 UTC,5652948553137053553,"""active""",2,"""America/New_Yo…",2023-01-18 05:00:00 UTC,2023-01-19 04:59:00 UTC
2023-01-18 05:03:03.523237 UTC,5652948553137053553,"""active""",2,"""America/New_Yo…",2023-01-18 05:00:00 UTC,2023-01-19 04:59:00 UTC
2023-01-18 05:04:03.523237 UTC,5652948553137053553,"""active""",2,"""America/New_Yo…",2023-01-18 05:00:00 UTC,2023-01-19 04:59:00 UTC
2023-01-18 05:05:03.523237 UTC,5652948553137053553,"""active""",2,"""America/New_Yo…",2023-01-18 05:00:00 UTC,2023-01-19 04:59:00 UTC
2023-01-18 05:06:03.523237 UTC,5652948553137053553,"""active""",2,"""America/New_Yo…",2023-01-18 05:00:00 UTC,2023-01-19 04:59:00 UTC
2023-01-18 05:07:03.523237 UTC,5652948553137053553,"""active""",2,"""America/New_Yo…",2023-01-18 05:00:00 UTC,2023-01-19 04:59:00 UTC
2023-01-18 05:08:03.523237 UTC,5652948553137053553,"""active""",2,"""America/New_Yo…",2023-01-18 05:00:00 UTC,2023-01-19 04:59:00 UTC


In [72]:
import datetime

x = []
y = []
for value in list(test.keys()):
    active = test[value].lazy().filter(pl.col("status") == "active")
    # print(active.collect())
    # current_date = active.select('timestamp_utc').head(1)['timestamp_utc'][0]
    # till_date = current_date - datetime.timedelta(days=1)
    inactive = test[value].lazy().filter(pl.col("status") == "inactive")
    # converted_current_date = test[value].select(pl.max("timestamp_utc"))
    converted_current_date = test[value].select(pl.max("timestamp_utc"))['timestamp_utc'][0]
    till_date = converted_current_date - datetime.timedelta(days=7)
    active_hours = active.lazy().filter(
        (pl.col("timestamp_utc") > till_date)
        & (pl.col("timestamp_utc") <= converted_current_date)
    )
    # print(active.select(
    #   pl.count()
    # ))
    # print(inactive.select(
    #   pl.count()
    # ))
    # current_date = inactive.select('timestamp_utc').head(1)['timestamp_utc'][0]
    # till_date = current_date - datetime.timedelta(days=1)
    inactive_hours = inactive.lazy().filter(
        (pl.col("timestamp_utc") > till_date)
        & (pl.col("timestamp_utc") <= converted_current_date)
    )

    active_hours_value = active_hours.select(pl.count())

    inactive_hours_value = inactive_hours.select(pl.count())
    # x.append(pl.first(active_hours_value.collect()['count']))
    # y.append(pl.first(inactive_hours_value.collect()['count']))
    # print(pl.first(active_hours_value.collect()["count"] / (60 * 24)))
    # print(pl.first(inactive_hours_value.collect()["count"] / (60 * 24)))

    # print(active_hours_value/(60*24), inactive_hours_value/(60))
# active.filter(
#   pl.col('timestamp_utc')
# )

In [64]:
df = filter_non_247.collect().to_pandas()
df.sort_values(by='store_id', ascending=False).sort_values(by='timestamp_utc', ascending=False).head()

,store_id,status,timestamp_utc,day,timezone_str,start_time_local,end_time_local
0,85496058573776375,active,2023-01-25 18:13:22.479220+00:00,2,America/New_York,2023-01-25 16:00:00+00:00,2023-01-26 02:00:00+00:00
1,4285890795849559187,active,2023-01-25 18:13:22.336259+00:00,2,America/New_York,2023-01-25 16:00:00+00:00,2023-01-26 04:59:00+00:00
2,3981100817087735334,active,2023-01-25 18:13:22.185498+00:00,2,America/New_York,2023-01-25 16:00:00+00:00,2023-01-26 02:45:00+00:00
3,8588363503810828914,active,2023-01-25 18:13:19.061219+00:00,2,America/New_York,2023-01-25 16:00:00+00:00,2023-01-26 02:00:00+00:00
4,5912314869414264686,inactive,2023-01-25 18:13:18.946294+00:00,2,America/Chicago,2023-01-25 17:00:00+00:00,2023-01-26 05:00:00+00:00


In [ ]:
list_keys = list(seperated_df.keys())
new_dict = {}
for i in list_keys:
    sample_timezone = seperated_df[i].select("timezone_str").head(1)["timezone_str"][0]

    new_dict[i] = (
        seperated_df[i]
        .lazy()
        .select(
            pl.col(["store_id", "status", "timestamp_utc", "day"]),
            pl.col("start_time_local")
            .dt.replace_time_zone(sample_timezone)
            .dt.convert_time_zone("UTC"),
            pl.col("end_time_local")
            .dt.replace_time_zone(sample_timezone)
            .dt.convert_time_zone("UTC"),
            pl.col(["timezone_str"]),
        )
    )


In [ ]:
new = h.collect().select(pl.col(['timezone_str', 'store_id'])).to_pandas()

In [ ]:
h.select(
    pl.col("timezone_str").dt.replace_time_zone(
        new.loc[new["store_id"] == pl.col("store_id"), "timezone_str"].iloc[0]
    )
)